In [1]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [2]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# # filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

# _filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_perf_rank_dict =  'perf_rank_dict'  # store symbols from performance rank results
filename_pickled_r_all_ranks =  'r_all_ranks'  # list of top 100 most common symbols from performance rank results
filename_pickled_df_a = 'df_OHLCV_clean'  # df adjusted OHLCV, dropped symbols with no vol and close
filename_pickled_df_c = 'df_close_clean'  # df close, dropped symbols with no vol and close

verbose = False  # True prints more output

#################
# look_back_days = -250 * 60  # subset df iloc days
look_back_days = -250 * 6  # subset df iloc days, 6 years of data
#################

In [3]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [4]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_adjOHLCV[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

symbols with no volume:
Index(['AAWW', 'AMCR', 'AVT', 'BCI', 'BHF', 'BKR', 'CCI', 'CHD', 'CNC', 'CNXC',
       'ESAB', 'FCFS', 'GEN', 'HUBB', 'IBKR', 'JJC', 'PEAK', 'PFG', 'SBNY',
       'UA', 'VC', 'VNT', 'WH', 'XEL'],
      dtype='object')


In [5]:
df_dif = df_v - df_v.shift(periods=1)
rows, cols = np.where(df_dif == 0)
idx_same_volume = list(set(cols))
idx_same_volume.sort()
idx_same_volume
symbols_same_volume = df_v.columns[idx_same_volume]
print(f'symbols with same volume:\n{symbols_same_volume}')

symbols with same volume:
Index(['AAWW', 'ACM', 'ACN', 'ADP', 'AGCO', 'AMCR', 'AMED', 'AMG', 'AMGN',
       'ANSS',
       ...
       'VAC', 'VICR', 'VNDA', 'VRSK', 'VSAT', 'WERN', 'WEX', 'WTS', 'WTW',
       'XEL'],
      dtype='object', length=213)


In [6]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

symbols with NaN close:
Index(['AAWW', 'ABNB', 'AIRC', 'ALGM', 'ATCO', 'BHF', 'BJ', 'BOH', 'BRBR',
       'CARR', 'CDAY', 'CEG', 'CHK', 'CHX', 'CNXC', 'CR', 'CRWD', 'CTVA',
       'DDOG', 'DOW', 'DT', 'DTM', 'ENSG', 'ESAB', 'ETH-USD', 'ETRN', 'FOX',
       'FOXA', 'FRC', 'FYBR', 'GDS', 'GEHC', 'GFS', 'GO', 'GXO', 'HTH', 'IAA',
       'JBGS', 'JJC', 'KD', 'LCID', 'MP', 'MRNA', 'MTLS', 'NARI', 'NVST',
       'NVT', 'OGN', 'OTIS', 'PDD', 'PEB', 'PGNY', 'RIVN', 'SEDG', 'SGOV',
       'SHC', 'SITM', 'SJR', 'SRPT', 'SWAV', 'TROX', 'USAC', 'VEEV', 'VICI',
       'VNT', 'VSCO', 'WH', 'YETI', 'ZM', 'ZS'],
      dtype='object')


In [7]:
symbols_drop = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
print(f'combined symbols with no volume, same volume and no close, inculdes duplicate symbols: {len(symbols_drop)}')
symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
symbols_drop .sort()
df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
print(f'unique symbols dropped from df_a (adjOLHLV) and df_c (Close): {len(symbols_drop)}')

combined symbols with no volume, same volume and no close, inculdes duplicate symbols: 307
unique symbols dropped from df_a (adjOLHLV) and df_c (Close): 284


In [8]:
print(f'symbols with no volume:   {len(symbols_no_volume):>5,}')
print(f'symbols with same volume: {len(symbols_same_volume):>5,}')
print(f'symbols with no close:    {len(symbols_no_close):>5,}\n')
print(f'symbols total before drop:                                        {len(df_close.columns):>5,}')
print(f'unique symbols dropped from df OHLCVA (df_a) and df Close (df_c): {len(symbols_drop):>5,}\n')
print('                                          symbols     rows')
print(f'df adjOHLCV (df_a) after dropped symbols:   {len(df_a.columns)/5:>5,.0f}    {len(df_a):>5,}')
print(f'df Close (df_c) after dropped symbols:      {len(df_c.columns):>5,}    {len(df_c):>5,}')


symbols with no volume:      24
symbols with same volume:   213
symbols with no close:       70

symbols total before drop:                                        1,587
unique symbols dropped from df OHLCVA (df_a) and df Close (df_c):   284

                                          symbols     rows
df adjOHLCV (df_a) after dropped symbols:   1,303    1,500
df Close (df_c) after dropped symbols:      1,303    1,500


In [9]:
pickle_dump(df_a, path_data_dump, filename_pickled_df_a)
print(f'pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_a}')
pickle_dump(df_c, path_data_dump, filename_pickled_df_c)
print(f'pickled df Close after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_c}')

pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCV_clean
pickled df Close after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_close_clean


In [10]:
from myUtils import list_dump

f_symbols_df_close_clean = 'symbols_df_close_clean.csv'  # symbols text file
symbols_df_c = list(df_c)  # column names in df_c
list_dump(symbols_df_c, path_data_dump, f_symbols_df_close_clean)# df_c.columns.to_csv(f_symbols_df_close_clean)

In [11]:
df_a.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2023-05-08  134.009995  134.210007  131.750000  132.449997  1005300.0   
2023-05-09  128.500000  128.500000  125.360001  127.349998  3437100.0   
2023-05-10  128.710007  128.899994  126.160004  128.059998  1255700.0   
2023-05-11  127.169998  127.699997  125.470001  127.660004  1580600.0   
2023-05-12  128.679993  128.679993  126.525002  127.489998  1395229.0   

                   AA                                              ...   ZUMZ  \
                 Open       High        Low      Close     Volume  ...   Open   
Date                                                               ...          
2023-05-08  37.959999  38.200001  36.689999  36.820000  3750400.0  ...  16.58   
2023-05-09  36.099998  36.919998  35.709999  36.509998  3445000.0  ...  16.32   
2023-05-10  37.139999  37.220001  35.380001  36.139999  2961200.0  ...  16.49   
2023-05-11  35.400002  35.930000  35.060001  35.330002  3272400.0  ...  15.91   
2023-05-12  35.439999  35.500000  35.029999  35.369999  1955508.0  ...  16.24   

                                                         ZWS             \
                 High    Low      Close    Volume       Open       High   
Date                                                                      
2023-05-08  17.340000  16.43  16.530001  314200.0  21.570000  21.680000   
2023-05-09  16.420000  16.01  16.139999  194300.0  21.360001  21.389999   
2023-05-10  16.490000  15.59  15.860000  245300.0  21.500000  21.530001   
2023-05-11  16.370001  15.82  16.160000  225300.0  21.110001  21.110001   
2023-05-12  16.275000  15.82  15.990000  203601.0  21.010000  21.219999   

                                             
                  Low      Close     Volume  
Date                                         
2023-05-08  21.160000  21.480000   910000.0  
2023-05-09  21.110001  21.260000   822600.0  
2023-05-10  21.100000  21.350000  1432800.0  
2023-05-11  20.700001  20.910000  1022600.0  
2023-05-12  20.760000  20.969999   766412.0  

[5 rows x 6515 columns]

In [12]:
df_c.tail()

,A,AA,AAL,AAP,AAPL,AB,ABB,ABBV,ABC,ABM,...,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2023-05-08,132.449997,36.820000,14.36,125.750000,173.500000,34.410000,36.139999,147.449997,167.728699,42.320000,...,29.709999,137.440002,277.859985,65.519997,47.380001,24.26,28.299999,184.869995,16.530001,21.480000
2023-05-09,127.349998,36.509998,14.47,125.519997,171.770004,33.889999,36.570000,146.830002,167.768570,42.590000,...,29.270000,136.050003,270.160004,64.970001,46.619999,24.42,28.350000,184.369995,16.139999,21.260000
2023-05-10,128.059998,36.139999,14.34,125.800003,173.559998,34.860001,36.459999,146.419998,169.014999,42.799999,...,27.980000,137.419998,271.769989,65.489998,46.520000,23.75,28.750000,185.139999,15.860000,21.350000
2023-05-11,127.660004,35.330002,14.07,124.169998,173.750000,34.779999,36.730000,146.589996,171.440002,43.240002,...,29.580000,136.690002,265.279999,63.750000,45.020000,22.68,28.719999,185.119995,16.160000,20.910000
2023-05-12,127.489998,35.369999,14.02,122.860001,172.570007,34.169998,36.930000,147.149994,170.990005,43.380001,...,29.299999,136.750000,266.160004,62.709999,44.139999,22.43,28.360001,183.619995,15.990000,20.969999
